# 기본 패키지 설치

In [ ]:
# 가상환경을 만든 뒤 진행
pip install langchain-community pymysql   # DB 연결은 나중에 필요, 지금은 로드 용도로도 필요 없음
# 문서 타입별 로더가 필요하면 아래 패키지도 설치
pip install unstructured                 # docx, html 등 Unstructured 계열 로더
pip install PyPDF2                       # pdf 로더 (PyPDFLoader 내부에서 사용)
pip install pandas                       # CSV 로더용

In [11]:
import langchain_community
print(langchain_community.__version__)

0.3.31


# 임포트

In [12]:
import os
from langchain_community.llms import Ollama
from langchain.chains.summarize import load_summarize_chain
from langchain_core.prompts import PromptTemplate
import pymysql

# 파일주소 입력

In [13]:
# 파일 정보
file_path = "fileList/IT기사.docx"
file_name = os.path.basename(file_path)
file_location = os.path.abspath(file_path)

# 파일 불러오기 전체 코드 (Jupyter/스크립트 겸용)

In [14]:
def load_document(file_path: str):
    # 파일 확장자에 따라 적절한 LangChain DocumentLoader로 문서를 불러옵니다.
    ext = os.path.splitext(file_path)[1].lower()
    if ext == ".docx":
        from langchain_community.document_loaders import UnstructuredWordDocumentLoader
        loader = UnstructuredWordDocumentLoader(file_path)
    elif ext == ".pdf":
        from langchain_community.document_loaders import PyPDFLoader
        loader = PyPDFLoader(file_path)
    elif ext == ".csv":
        from langchain_community.document_loaders import CSVLoader
        loader = CSVLoader(file_path, encoding="utf-8")
    elif ext == ".txt":
        from langchain_community.document_loaders import TextLoader
        loader = TextLoader(file_path, encoding="utf-8")
    elif ext in (".html", ".htm"):
        from langchain_community.document_loaders import UnstructuredFileLoader
        loader = UnstructuredFileLoader(file_path, mode="elements")
    else:
        from langchain_community.document_loaders import UnstructuredFileLoader
        loader = UnstructuredFileLoader(file_path, mode="elements")
    docs = loader.load()
    return docs


# 파일 로드 확인

In [15]:
# 파일이 실제로 존재하는지 체크
if not os.path.isfile(file_path):
    print(f"❌ 파일이 존재하지 않습니다: {file_path}")
else:
    try:
        docs = load_document(file_path)
        print(f"✅ 로드 완료 – 총 {len(docs)} 개 청크/페이지")
        if docs:
            # 모든 청크 출력
            for i, doc in enumerate(docs):
                print(f"\n--- 문서 청크 {i+1} ---\n{doc.page_content}\n")
        else:
            print("문서에 내용이 없습니다.")
    except Exception as e:
        print(f"❌ 문서 로드 실패: {e}")

❌ 문서 로드 실패: cannot import name 'is_cjk_uncommon' from 'charset_normalizer.utils' (/home/alpaco/anaconda3/envs/file_upload/lib/python3.10/site-packages/charset_normalizer/utils.py)


# 프롬프트 템플릿

In [ ]:
PROMPT_TEMPLATE = """
다음 문서 내용을 분석하여 아래 항목을 한글로 한 줄씩 추출해줘.

title: 문서의 제목을 한 줄로,
summary: 전체 내용을 1000자 이내로 줄거리처럼 요약해줘. (띄어쓰기 포함 1000자 이하, 너무 짧게 쓰지 말고 최대한 자세히)
keywords: 문서의 핵심 단어를 50개 정도, 쉼표(,)로 구분해서 한 줄로 나열해줘.

아래 형식으로만 출력해줘.
title: ...
summary: ...
keywords: ...

문서 내용:
"{text}"
"""

CUSTOM_PROMPT = PromptTemplate(template=PROMPT_TEMPLATE, input_variables=["text"])


In [74]:
# 3. LLM 요약 함수
def summarize_with_llm(docs):
    llm = Ollama(model="exaone3.5:2.4b")
    summarize_chain = load_summarize_chain(
        llm,
        chain_type="stuff",
        prompt=CUSTOM_PROMPT
    )
    result = summarize_chain.invoke(docs)
    return result["output_text"]

In [ ]:
# 4. LLM 결과 파싱 함수
import re
def parse_llm_output(output_text):
    # 정규식으로 각 항목 추출
    title = re.search(r"title\s*:\s*(.*)", output_text)
    summary = re.search(r"summary\s*:\s*(.*)", output_text)
    keywords = re.search(r"keywords\s*:\s*(.*)", output_text)
    return {
        "title": title.group(1).strip() if title else "",
        "summary": summary.group(1).strip() if summary else "",
        "keywords": keywords.group(1).strip() if keywords else "",
    }

In [ ]:
# 6. 실행
if not os.path.isfile(file_path):
    print(f"❌ 파일이 존재하지 않습니다: {file_path}")
else:
    docs = load_document(file_path)
    print(f"✅ 문서 로드 완료. 청크 수: {len(docs)}")
    llm_output = summarize_with_llm(docs)
    print("\n--- LLM 출력 ---\n", llm_output)
    parsed = parse_llm_output(llm_output)
    print("\n--- 최종 결과 ---")
    print("title:", parsed["title"])
    print("summary:", parsed["summary"])
    print("keywords:", parsed["keywords"])
    print("file_location:", file_location)
    print("file_name:", file_name)

✅ 문서 로드 완료. 청크 수: 14

--- LLM 출력 ---
 doctype html
html lang="ko">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>문화상품권 온라인 신청 - 신문 구독 지원</title>
</head>
<body>
    <h1>문화상품권 온라인 신청</h1>
    <h2>신문 구독 지원 안내</h2>
    
    <h3>신청 안내</h3>
    <ul>
        <li><strong>신청대상</strong>: 기초생활수급자, 차상위계층, 장애인으로 확인된 분들</li>
        <li><strong>신청기간</strong>: 매년 1월 2일부터 ~ 12월 31일까지</li>
        <li><strong>신청방법</strong>: 정부24 홈페이지 또는 모바일 앱을 통해 신청 가능</li>
        <li><strong>문화누리카드 소지 여부 무관</strong>: 문화상품권 소지 여부와 관계없이 신청 가능</li>
        <li><strong>신청 후 확인</strong>: MY GOV 메뉴에서 신청 완료 상태 확인</li>
    </ul>
    
    <h3>자주 묻는 질문</h3>
    <section>
        <h4>Q1</h4>
        <p><strong>A1</strong> - 신청 결과는 신청수요 및 배달지역 사정에 따라 결정되며, 2순위 희망 신문으로 선정될 수 있습니다.</p>
        <p><strong>Q1</strong> - <strong>A2</strong> 신청하면 모두 선정이 될 수 있습니다.</p>
        <h4>Q2</h4>
        <p><strong>A3</strong> - 2025년 1월 2일부터 신청 가능합니다.</p>
        <p><str

# Mysql DB에 저장하기

In [54]:
# 1. DB 접속 정보
DB_CONFIG = {
    'host': 'localhost', 
    'user': 'admin', 
    'password': '1qazZAQ!', 
    'db': 'final', 
    'charset': 'utf8mb4'
}

In [ ]:
# 2. 변수선언
title = parsed["title"]
summary = parsed["summary"]
keywords = parsed["keywords"]
# file_location, file_name은 위에서 이미 선언됨

In [ ]:
# 3. DB에 INSERT
try:
    conn = pymysql.connect(**DB_CONFIG)
    with conn.cursor() as cursor:
        sql = """
        INSERT INTO documents
        (title, summary, keywords, file_location, file_name, created_at)
        VALUES (%s, %s, %s, %s, %s, NOW())
        """
        cursor.execute(sql, (title, summary, keywords, file_location, file_name))
    conn.commit()
    print("✅ DB 저장 완료!")
except Exception as e:
    print("❌ DB 저장 실패:", e)
finally:
    if 'conn' in locals():
        conn.close()

✅ DB 저장 완료!
